Código para a obter a lista de tickers que já participaram do S&P 500.

Listas Iniciais

In [ ]:
import pandas as pd
import requests

# URL da página da Wikipedia
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)

# Ler tabelas
tables = pd.read_html(response.text, header=[0, 1])  # <- pega os dois níveis do cabeçalho

# Empresas atuais
df_current = tables[0][["Symbol", "Security"]].copy()
df_current.columns = ["Ticker", "Security"]
df_current["Status"] = "current"

# Histórico de mudanças (com MultiIndex)
df_changes = tables[1]

# Achatar os nomes de coluna (ex: ("Added","Ticker") -> "Added_Ticker")
df_changes.columns = ['_'.join(col).strip() for col in df_changes.columns.values]

# "Added" -> Ticker e Security
df_added = df_changes[["Added_Ticker", "Added_Security"]].copy()
df_added.columns = ["Ticker", "Security"]
df_added["Status"] = "added"

# "Removed" -> Ticker e Security
df_removed = df_changes[["Removed_Ticker", "Removed_Security"]].copy()
df_removed.columns = ["Ticker", "Security"]
df_removed["Status"] = "removed"

# Concatenar tudo
df_all = pd.concat([df_current, df_added, df_removed], ignore_index=True)

# Remover duplicados
df_all = df_all.dropna(subset=["Ticker"]).drop_duplicates()
df_all = df_all.dropna(subset=["Ticker"]).drop_duplicates(subset=["Ticker"], keep="first")
df_all.loc[df_all["Status"] != "current", "Status"] = "removed"
df_all = df_all.sort_values("Ticker").reset_index(drop=True)
df_all.to_csv("sp500_historical_list.csv", index=False)

print(df_all.head(20))

Filtrar tickers válidos

In [ ]:
import yfinance as yf
import pandas as pd

# Ler CSV
df_all = pd.read_csv("sp500_historical_list.csv")

# Lista de tickers
tickers = df_all["Ticker"].tolist()

# Função para testar se o ticker é válido
def check_ticker(ticker):
    try:
        data = yf.Ticker(ticker).history(period="1d")
        return not data.empty
    except Exception:
        return False

# Verificar todos os tickers
validity = {ticker: check_ticker(ticker) for ticker in tickers}

# Criar DataFrame com resultados
df_validity = pd.DataFrame(list(validity.items()), columns=["Ticker", "Valid"])
print(df_validity.head())

# Filtrar apenas tickers válidos
valid_tickers = df_validity[df_validity["Valid"]]["Ticker"].tolist()

df_validity.to_csv("tickers_validity.csv", index=False)
pd.DataFrame(valid_tickers, columns=["Ticker"]).to_csv("valid_tickers.csv", index=False)

print(f"Número de tickers válidos: {len(valid_tickers)}")

Encontrar informações sobre os tickers inválidos

In [ ]:
import pandas as pd
import yfinance as yf

# Ler CSVs
df_all = pd.read_csv("sp500_historical_list.csv")
df_validity = pd.read_csv("tickers_validity.csv")

# Tickers válidos e inválidos
valid_tickers = df_validity[df_validity["Valid"] == True]["Ticker"].tolist()
invalid_tickers = df_validity[df_validity["Valid"] == False]["Ticker"].tolist()

print(f"Tickers válidos: {len(valid_tickers)}")
print(f"Tickers inválidos: {len(invalid_tickers)}")

# Função para tentar buscar informações de tickers inválidos
def get_info(ticker):
    try:
        t = yf.Ticker(ticker)
        info = t.info
        if info and "longName" in info:
            return {"Ticker": ticker, "Found": True, "Name": info.get("longName"), "Exchange": info.get("exchange")}
        else:
            return {"Ticker": ticker, "Found": False, "Name": None, "Exchange": None}
    except Exception:
        return {"Ticker": ticker, "Found": False, "Name": None, "Exchange": None}

# Rodar análise nos inválidos
results = [get_info(ticker) for ticker in invalid_tickers]

# Criar DataFrame com resultados
df_invalid_analysis = pd.DataFrame(results)
df_invalid_analysis.to_csv("tickers_invalid_analysis.csv", index=False)

print(df_invalid_analysis.head())

obter dados dos tickers válidos

In [3]:
import yfinance as yf
import pandas as pd
import time

# Ler lista de tickers válidos
df_tickers = pd.read_csv("valid_tickers.csv")
tickers = df_tickers["Ticker"].tolist()

# DataFrames para armazenar os dados
df_prices = pd.DataFrame()
df_info = pd.DataFrame()
df_dividends = pd.DataFrame()
df_splits = pd.DataFrame()

# Função para extrair informações fundamentais
def get_info_dict(ticker):
    try:
        t = yf.Ticker(ticker)
        
        # Preços históricos diários
        hist = t.history(period="max", interval="1d").reset_index()
        hist["Ticker"] = ticker
        global df_prices
        df_prices = pd.concat([df_prices, hist], ignore_index=True)
        
        # Dividendos
        divs = t.dividends.reset_index()
        divs["Ticker"] = ticker
        global df_dividends
        df_dividends = pd.concat([df_dividends, divs], ignore_index=True)
        
        # Splits
        splits = t.splits.reset_index()
        splits["Ticker"] = ticker
        global df_splits
        df_splits = pd.concat([df_splits, splits], ignore_index=True)
        
        # Informações fundamentais
        info = t.info
        info["Ticker"] = ticker
        global df_info
        df_info = pd.concat([df_info, pd.DataFrame([info])], ignore_index=True)
        
        print(f"✅ {ticker} coletado")
    except Exception as e:
        print(f"❌ Erro em {ticker}: {e}")

# Loop por todos os tickers
for i, ticker in enumerate(tickers):
    get_info_dict(ticker)
    # Delay para evitar bloqueio do Yahoo
    time.sleep(0.5)
    if (i+1) % 50 == 0:
        print(f"{i+1}/{len(tickers)} tickers processados")

# Salvar em CSV
df_prices.to_csv("yf_prices.csv", index=False)
df_info.to_csv("yf_info.csv", index=False)
df_dividends.to_csv("yf_dividends.csv", index=False)
df_splits.to_csv("yf_splits.csv", index=False)

print("✅ Todos os dados salvos!")

✅ A coletado


C:\Users\João Paulo Bussi\AppData\Local\Temp\ipykernel_12892\3221935595.py:42: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_info = pd.concat([df_info, pd.DataFrame([info])], ignore_index=True)


✅ AA coletado
✅ AAL coletado
✅ AAP coletado
✅ AAPL coletado
✅ ABBV coletado
✅ ABNB coletado


C:\Users\João Paulo Bussi\AppData\Local\Temp\ipykernel_12892\3221935595.py:42: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_info = pd.concat([df_info, pd.DataFrame([info])], ignore_index=True)


✅ ABT coletado


...

✅ WTW coletado


C:\Users\João Paulo Bussi\AppData\Local\Temp\ipykernel_12892\3221935595.py:42: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_info = pd.concat([df_info, pd.DataFrame([info])], ignore_index=True)


✅ WU coletado
✅ WY coletado
✅ WYNN coletado
✅ XEL coletado
✅ XOM coletado
✅ XRAY coletado


C:\Users\João Paulo Bussi\AppData\Local\Temp\ipykernel_12892\3221935595.py:42: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_info = pd.concat([df_info, pd.DataFrame([info])], ignore_index=True)


✅ XRX coletado
✅ XYL coletado
✅ XYZ coletado
✅ YUM coletado
✅ ZBH coletado
✅ ZBRA coletado
✅ ZION coletado
✅ ZTS coletado
✅ Todos os dados salvos!


In [ ]:
import yfinance as yf
import pandas as pd

# Lista de índices globais
tickers = {
    "S&P500": "^GSPC",
    "Nasdaq": "^IXIC",
    "Ouro": "GC=F",
    "VIX": "^VIX"
}

dados_indices = {}

for nome, ticker in tickers.items():
    dados = yf.download(ticker, start="1900-01-01", interval="1mo")  # máximo disponível
    dados["Nome"] = nome
    dados_indices[nome] = dados

# Unir em um DataFrame multi-índice (nome, data)
indices_df = pd.concat(dados_indices, names=["Indice", "Data"])